#  Introduction

Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

# Problem Statement

Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:

* **sales**
* **menu**
* **members**

# Table 1: sales

The sales table captures all customer_id level purchases with an corresponding order_date and product_id information for when and what menu items were ordered.


| customer_id | order_date  | product_id |
|-------------|-------------|------------|
| A           | 2021-01-01  | 1          |
| A           | 2021-01-01  | 2          |
| A           | 2021-01-07  | 2          |
| A           | 2021-01-10  | 3          |
| A           | 2021-01-11  | 3          |
| A           | 2021-01-11  | 3          |
| B           | 2021-01-01  | 2          |
| B           | 2021-01-02  | 2          |
| B           | 2021-01-04  | 1          |
| B           | 2021-01-11  | 1          |
| B           | 2021-01-16  | 3          |
| B           | 2021-02-01  | 3          |
| C           | 2021-01-01  | 3          |
| C           | 2021-01-01  | 3          |
| C           | 2021-01-07  | 3          |


# Table 2: menu

The menu table maps the product_id to the actual product_name and price of each menu item.

| product_id | product_name | price |
|------------|--------------|-------|
| 1          | sushi        | 10    |
| 2          | curry        | 15    |
| 3          | ramen        | 12    |


# Table 3: members

The final members table captures the join_date when a customer_id joined the beta version of the Danny’s Diner loyalty program.
| customer_id | join_date  |
|-------------|------------|
| A           | 2021-01-07 |
| B           | 2021-01-09 |


## Importing the Database in MySQL

In [2]:
import mysql.connector
from mysql.connector import errorcode

In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [17]:
mydb= mysql.connector.connect(host='localhost',user="root", password="Abcd@1234")
cursor=mydb.cursor()


In [56]:
if mydb.is_connected():
    print('yes')

In [19]:
  # Create Database
query ='''CREATE DATABASE IF NOT EXISTS dannys_diner;
USE dannys_diner;

CREATE TABLE sales (
  customer_id VARCHAR(1),
  order_date DATE,
  product_id INT
);

INSERT INTO sales
  (customer_id, order_date, product_id)
VALUES
  ('A', '2021-01-01', 1),
  ('A', '2021-01-01', 2),
  ('A', '2021-01-07', 2),
  ('A', '2021-01-10', 3),
  ('A', '2021-01-11', 3),
  ('A', '2021-01-11', 3),
  ('B', '2021-01-01', 2),
  ('B', '2021-01-02', 2),
  ('B', '2021-01-04', 1),
  ('B', '2021-01-11', 1),
  ('B', '2021-01-16', 3),
  ('B', '2021-02-01', 3),
  ('C', '2021-01-01', 3),
  ('C', '2021-01-01', 3),
  ('C', '2021-01-07', 3);

CREATE TABLE menu (
  product_id INT,
  product_name VARCHAR(5),
  price INT
);

INSERT INTO menu
  (product_id, product_name, price)
VALUES
  (1, 'sushi', 10),
  (2, 'curry', 15),
  (3, 'ramen', 12);

CREATE TABLE members (
  customer_id VARCHAR(1),
  join_date DATE
);

INSERT INTO members
  (customer_id, join_date)
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');
'''
cursor.execute(query)
cursor.close()
mydb.close()

# CASE STUDY PROBLEMS

In [20]:
if mydb.is_connected():
    print('yes')
else:
    mydb= mysql.connector.connect(host='localhost',user="root", password="Abcd@1234")
    cursor=mydb.cursor()

In [21]:
cursor.execute("USE dannys_diner")
query ="SELECT * FROM sales"
cursor.execute(query)

results = cursor.fetchall()
for row in results:
    print(row)


('A', datetime.date(2021, 1, 1), 1)
('A', datetime.date(2021, 1, 1), 2)
('A', datetime.date(2021, 1, 7), 2)
('A', datetime.date(2021, 1, 10), 3)
('A', datetime.date(2021, 1, 11), 3)
('A', datetime.date(2021, 1, 11), 3)
('B', datetime.date(2021, 1, 1), 2)
('B', datetime.date(2021, 1, 2), 2)
('B', datetime.date(2021, 1, 4), 1)
('B', datetime.date(2021, 1, 11), 1)
('B', datetime.date(2021, 1, 16), 3)
('B', datetime.date(2021, 2, 1), 3)
('C', datetime.date(2021, 1, 1), 3)
('C', datetime.date(2021, 1, 1), 3)
('C', datetime.date(2021, 1, 7), 3)


**1. What is the total amount each customer spent at the restaurant?**

In [9]:
cursor.execute(
'''SELECT s.customer_id, 
          SUM(m.price) AS total_amount_spent
   FROM sales s
   JOIN menu m ON s.product_id = m.product_id
   GROUP BY s.customer_id'''
)
results = cursor.fetchall()

for row in results:
    customer_id, total_amount_spent = row
    total_amount_spent = int(total_amount_spent)  # Convert Decimal to float
    print(customer_id, total_amount_spent)    

A 76
B 74
C 36


**2. How many days has each customer visited the restaurant?**

In [10]:
cursor.execute('''SELECT customer_id,
                         COUNT(DISTINCT order_date) AS days_visited
                  FROM sales
                  GROUP BY customer_id
''')
results = cursor.fetchall()

for row in results:
    customer_id, days_visited = row
    days_visited = int(days_visited)  # Convert Decimal to float
    print(customer_id, days_visited)

A 4
B 6
C 2


**3. What was the first item from the menu purchased by each customer?**


In [11]:
cursor.execute('''
WITH CTE AS 
(
 SELECT customer_id, 
        MIN(order_date) AS first_visit_date,
        MIN(product_id) AS first_purchased_item_id
 FROM sales
 GROUP BY customer_id
 )
 
SELECT customer_id,
       m.product_name
FROM cte 
JOIN menu m ON cte.first_purchased_item_id=m.product_id
''')
results = cursor.fetchall()
    
for row in results:
    print(row)

('A', 'sushi')
('B', 'sushi')
('C', 'ramen')


**4. What is the most purchased item on the menu and how many times was it purchased by all customers?**

In [12]:
cursor.execute('''
WITH CTE AS
(
 SELECT product_id, 
        COUNT(*) AS purchase_count
 FROM sales
 GROUP BY product_id
 ORDER BY purchase_count DESC
 LIMIT 1
 )

SELECT m.product_name, 
       cte.purchase_count
FROM cte 
JOIN menu m ON cte.product_id=m.product_id
''')
results = cursor.fetchall()
    
for row in results:
    print(row)

('ramen', 8)


**5. Which item was the most popular for each customer?**

In [13]:
cursor.execute('''
WITH CTE AS (SELECT s.customer_id, 
                    m.product_name, 
                    COUNT(*) AS purchase_count
FROM sales s
JOIN menu m ON s.product_id = m.product_id
GROUP BY s.customer_id, s.product_id, m.product_name
ORDER BY customer_id, purchase_count desc)

SELECT customer_id, 
       product_name
FROM 
(
 SELECT customer_id, product_name, dense_rank() over(partition by customer_id order by purchase_count desc) as rnk
 FROM CTE
 ) t
WHERE rnk= 1


''')
results = cursor.fetchall()
    
for row in results:
    print(row)

('A', 'ramen')
('B', 'curry')
('B', 'sushi')
('B', 'ramen')
('C', 'ramen')


'B' did not have any favourites here.

**6.Which item was purchased first by the customer after they became a member?**

In [22]:
cursor.execute('''WITH CTE AS(
                                SELECT m.customer_id,
                                       u.product_name, 
                                       dense_rank() OVER(PARTITION BY m.customer_id Order BY s.order_date) as rnk
                                FROM sales s
                                JOIN members m ON s.customer_id = m.customer_id JOIN menu u ON s.product_id=u.product_id
                                WHERE s.order_date >= m.join_date 
                                )

                  SELECT customer_id,product_name
                  FROM CTE
                  WHERE rnk=1
                ''')
results = cursor.fetchall()
    
for row in results:
    print(row)

('A', 'curry')
('B', 'sushi')


**7. Which item was purchased just before the customer became a member?**

In [37]:
cursor.execute('''WITH CTE AS (
                                SELECT m.customer_id, 
                                       s.product_id, 
                                       u.product_name, 
                                       s.order_date, 
                                       DENSE_RANK() OVER (PARTITION BY m.customer_id ORDER BY s.order_date) as rnk
                                FROM sales s
                                JOIN members m ON s.customer_id = m.customer_id JOIN menu u ON s.product_id=u.product_id
                                WHERE s.order_date < m.join_date
                                ORDER BY m.customer_id
                                )

                  SELECT *
                  FROM CTE 
                  WHERE rnk=1

                ''')
results = cursor.fetchall()
    
for row in results:
    print(row)

('A', 1, 'sushi', datetime.date(2021, 1, 1), 1)
('A', 2, 'curry', datetime.date(2021, 1, 1), 1)
('B', 2, 'curry', datetime.date(2021, 1, 1), 1)


'A' ordered two items just before becoming a member

**8. What is the total items and amount spent for each member before they became a member?**

In [40]:
cursor.execute('''SELECT m.customer_id, 
                         COUNT(s.product_id) as total_items, 
                         SUM(menu.price) as total_amount_spent
                  FROM sales s
                  JOIN menu ON s.product_id = menu.product_id
                  JOIN members m ON s.customer_id = m.customer_id
                  WHERE s.order_date < m.join_date
                  GROUP BY m.customer_id
                  ORDER BY m.customer_id
                ''')
results = cursor.fetchall()
    
for row in results:
    customer_id, total_items, total_amount_spent = row
    total_amount_spent= int(total_amount_spent)  # Convert Decimal to float
    print(customer_id, total_items, total_amount_spent)

A 2 25
B 3 40


**9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?**

In [46]:
cursor.execute('''SELECT s.customer_id,
                         SUM(CASE WHEN m.product_name = 'sushi' THEN m.price * 20 ELSE m.price * 10 END) AS total_points
                  FROM sales s
                  JOIN menu m ON s.product_id = m.product_id
                  GROUP BY s.customer_id;
                ''')
results = cursor.fetchall()
    
for row in results:
    customer_id, total_points = row
    total_points = int(total_points)  # Convert Decimal to float
    print(customer_id, total_points)

A 860
B 940
C 360


**10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?**

In [53]:
cursor.execute('''WITH CTE AS (
                              SELECT customer_id, 
                                     join_date,
                                     join_date + 6 AS valid_date,
                                     DATE_SUB(LAST_DAY('2021-01-31'), INTERVAL 1 DAY)  AS last_date
                              FROM members
                                )

                    SELECT  s.customer_id, 
                            SUM(CASE
                                    WHEN m.product_name = 'sushi' THEN 2 * 10 * m.price
                                    WHEN s.order_date BETWEEN dates.join_date AND dates.valid_date THEN 2 * 10 * m.price
                                    ELSE 10 * m.price 
                                    END
                                )AS points
                    FROM sales s
                    JOIN CTE AS dates
                    ON s.customer_id = dates.customer_id
                    AND s.order_date <= dates.last_date
                    JOIN menu m
                    ON s.product_id = m.product_id
                    GROUP BY s.customer_id 
                    ORDER BY s.customer_id
''')
results = cursor.fetchall()
    
for row in results:
    customer_id, points = row
    points = int(points)  # Convert Decimal to float
    print(customer_id, points)

A 1370
B 820


In [55]:
cursor.close()
mydb.close()
mydb.commit()